requirements
make a virtual environment with conda.. (for this the virtual environment is "torch_env")
download pytorch and all other dependencies in that environment.
start making your nn model.

In [1]:
"""
here what we are trying to do is creating a multiclass classification for the gesture
detection. there are altogether 8 gestures labelled from 0-7 which are used to control 
the drone ultimately. a gesture detection model with 97% accuracy in obtained.

"""

import torch
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt


#setting up device agnostic code
device="cuda" if torch.cuda.is_available() else "cpu"

print(device)

# one of the ways to read csv file in colab is to upload the csv file in github and do like this
# url='https://raw.githubusercontent.com/Bikash9841/gestureD/main/keypointUP.csv'

# processed landmarks self made dataset
# url='https://raw.githubusercontent.com/Bikash9841/gestureD/main/keypoint.csv'


# this is from github dataset but for 8 gestures
url='https://media.githubusercontent.com/media/kinivi/tello-gesture-control/main/model/keypoint_classifier/keypoint.csv'

df=pd.read_csv(url)

df=shuffle(df,random_state=42)

# req_df = df[['0x', '0y', '1x', '1y', '2x', '2y', '3x', '3y', '4x', '4y', '5x', '5y', '6x', '6y', '7x', '7y', '8x', '8y', '9x', '9y', '10x', '10y', '11x', '11y',
                #  '12x', '12y', '13x', '13y', '14x', '14y', '15x', '15y', '16x', '16y', '17x', '17y', '18x', '18y', '19x', '19y', '20x', '20y']]

# X=req_df.values
# y=df['gesture'].values

# for github's 3 dataset
# req_df=df.iloc[:,1:]
# X=req_df.values
# y=df.iloc[:,0].values

# for github's 8 dataset
req_df=df.iloc[:,1:]
X=req_df.values
y=df.iloc[:,0].values


# for preprocessed dataset
# req_df=df.iloc[:,:42]
# X=req_df.values
# y=df.iloc[:,42].values

# print(X.shape,y.shape)

# print(X[:5,:])
# print(y[:5])


# turning data into tensors
X=torch.from_numpy(X).type(torch.float).to(device)
y=torch.from_numpy(y).type(torch.LongTensor).to(device)


print(X.shape,y.shape,X,y)

# to see the data in the graph uncomment all line below and comment X and y above


# X=df[['0x', '1x' , '2x', '3x', '4x', '5x', '6x', '7x', '8x', '9x', '10x','11x','12x','13x','14x','15x','16x','17x','18x','19x','20x']].values
# y=df[['0y', '1y' , '2y', '3y', '4y', '5y', '6y', '7y', '8y', '9y', '10y','11y','12y','13y','14y','15y','16y','17y','18y','19y','20y']].values
# g=df['gesture'].values
# # print(X.shape,y.shape)

# # print(df.head())


# print(g.shape,X.shape,y.shape)
# # print(req_df[:5])
# # print(g[0],X[0],y[0])


# plt.figure(figsize=(10, 7))
# plt.scatter(X[:,0],y[:,0],c=g,cmap=plt.cm.RdYlBu)




cpu
torch.Size([6189, 42]) torch.Size([6189]) tensor([[ 0.0000,  0.0000,  0.0585,  ...,  0.1122,  0.7659,  0.1463],
        [ 0.0000,  0.0000, -0.1863,  ..., -0.5735,  0.1814, -0.6667],
        [ 0.0000,  0.0000, -0.1559,  ..., -0.2634,  0.0376, -0.2366],
        ...,
        [ 0.0000,  0.0000, -0.4865,  ..., -0.0270,  0.1351, -0.0541],
        [ 0.0000,  0.0000, -0.2946,  ..., -0.5268,  0.2054, -0.4643],
        [ 0.0000,  0.0000,  0.2202,  ..., -0.8452,  0.1786, -0.9762]]) tensor([0, 0, 2,  ..., 5, 6, 0])


In [2]:
# splitting dataset into train and test set
from sklearn.model_selection import train_test_split
from torch import nn

X_train,X_test,y_train,y_test=train_test_split(X,
    y,
    test_size=0.2,
    random_state=42)

print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)


class GestureModel(nn.Module):
  def __init__(self,input_features,output_features,hidden_units=18):

    super().__init__()
    self.linear_layer_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_features), # how many classes are there?
        )
    
  def forward(self, x):
    return self.linear_layer_stack(x)

# create a instance of model and send it to the target device
model_1=GestureModel(42,8).to(device)

model_1



torch.Size([4951, 42]) torch.Size([1238, 42]) torch.Size([4951]) torch.Size([1238])


GestureModel(
  (linear_layer_stack): Sequential(
    (0): Linear(in_features=42, out_features=18, bias=True)
    (1): ReLU()
    (2): Linear(in_features=18, out_features=18, bias=True)
    (3): ReLU()
    (4): Linear(in_features=18, out_features=18, bias=True)
    (5): ReLU()
    (6): Linear(in_features=18, out_features=18, bias=True)
    (7): ReLU()
    (8): Linear(in_features=18, out_features=8, bias=True)
  )
)

In [3]:
# Create loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_1.parameters(), 
                            lr=0.1)

# Perform a single forward pass on the data (we'll need to put it to the target device for it to work)
model_1(X_train)[:5]


tensor([[ 0.1874,  0.0803,  0.1618, -0.1942, -0.0905,  0.2202,  0.0975, -0.0206],
        [ 0.1851,  0.0840,  0.1597, -0.1932, -0.0903,  0.2169,  0.0997, -0.0234],
        [ 0.1874,  0.0812,  0.1568, -0.1989, -0.0895,  0.2150,  0.1014, -0.0227],
        [ 0.1826,  0.0852,  0.1629, -0.1861, -0.0913,  0.2197,  0.1012, -0.0244],
        [ 0.1818,  0.0868,  0.1628, -0.1866, -0.0905,  0.2170,  0.0993, -0.0255]],
       grad_fn=<SliceBackward0>)

In [4]:
# Make prediction logits with model without actually training just to see the working of softmax 
y_logits = model_1(X_test)

print(y_logits)
# Perform softmax calculation on logits across dimension 1 to get prediction probabilities
y_pred_probs = torch.softmax(y_logits, dim=1) 

# print(y_logits[:5])
# print(y_pred_probs[:5])


# Which class does the model think is *most* likely at the index 0 sample?
print(y_pred_probs[0])

# just select the largest probabilities among 8 classses of 0th index.
print(torch.argmax(y_pred_probs[0]))



tensor([0.1408, 0.1275, 0.1375, 0.0971, 0.1072, 0.1459, 0.1294, 0.1146],
       grad_fn=<SelectBackward0>)
tensor(5)


-----------------------------------------LETS TRAIN OUR MODEL NOW.


In [5]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100 
    return acc

In [13]:
# Fit the model
torch.manual_seed(42)

# Set number of epochs
epochs = 1000

for epoch in range(epochs):
    ### Training
    model_1.train()

    # 1. Forward pass
    y_logits = model_1(X_train) # model outputs raw logits 
    y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1) # go from logits -> prediction probabilities -> prediction labels
    # print(y_logits)
    # 2. Calculate loss and accuracy
    loss = loss_fn(y_logits, y_train) 
    acc = accuracy_fn(y_true=y_train,
                      y_pred=y_pred)

    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backwards
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    ### Testing
    model_1.eval()
    with torch.inference_mode():
      # 1. Forward pass
      test_logits = model_1(X_test)
      test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
      # 2. Calculate test loss and accuracy
      test_loss = loss_fn(test_logits, y_test)
      test_acc = accuracy_fn(y_true=y_test,
                             y_pred=test_pred)

    # Print out what's happening
    if epoch % 10 == 0:
        print(f"Epoch: {epoch} | Loss: {loss:.5f}, Acc: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Acc: {test_acc:.2f}%")

Epoch: 0 | Loss: 0.61491, Acc: 81.05% | Test Loss: 0.63058, Test Acc: 81.10%
Epoch: 10 | Loss: 0.57218, Acc: 82.27% | Test Loss: 0.58724, Test Acc: 81.50%
Epoch: 20 | Loss: 0.56783, Acc: 81.74% | Test Loss: 0.60709, Test Acc: 80.13%
Epoch: 30 | Loss: 0.61617, Acc: 80.75% | Test Loss: 0.61039, Test Acc: 82.23%
Epoch: 40 | Loss: 0.53086, Acc: 83.28% | Test Loss: 0.54321, Test Acc: 82.39%
Epoch: 50 | Loss: 0.50285, Acc: 83.76% | Test Loss: 0.51387, Test Acc: 83.36%
Epoch: 60 | Loss: 0.65237, Acc: 76.55% | Test Loss: 0.83195, Test Acc: 68.74%
Epoch: 70 | Loss: 0.51633, Acc: 84.02% | Test Loss: 0.52334, Test Acc: 83.44%
Epoch: 80 | Loss: 0.46821, Acc: 84.37% | Test Loss: 0.47570, Test Acc: 84.25%
Epoch: 90 | Loss: 0.44119, Acc: 84.85% | Test Loss: 0.44781, Test Acc: 85.06%
Epoch: 100 | Loss: 0.42123, Acc: 86.18% | Test Loss: 0.42870, Test Acc: 84.98%
Epoch: 110 | Loss: 0.57712, Acc: 79.30% | Test Loss: 0.68085, Test Acc: 75.20%
Epoch: 120 | Loss: 0.45141, Acc: 85.38% | Test Loss: 0.45192, T

In [7]:
from sklearn.metrics import f1_score

# Make predictions
model_1.eval()
with torch.inference_mode():
    y_logits = model_1(X_test)

# View the first 10 predictions
y_logits[10:20]


# Turn predicted logits in prediction probabilities
y_pred_probs = torch.softmax(y_logits, dim=1)

# Turn prediction probabilities into prediction labels
y_preds = y_pred_probs.argmax(dim=1)

# f1 = F1Score(task="multiclass", num_classes=8)

print(f"f1 score: {f1_score(y_test,y_preds,average='macro')}")


# Compare first 10 model preds and test labels
print(f"Predictions: {y_preds[10:20]}\nLabels: {y_test[10:20]}")
print(f"Test accuracy: {accuracy_fn(y_true=y_test, y_pred=y_preds)}%")

f1 score: 0.6238366196949328
Predictions: tensor([1, 0, 0, 2, 1, 0, 1, 1, 2, 1])
Labels: tensor([7, 0, 0, 2, 1, 0, 1, 1, 2, 1])
Test accuracy: 81.01777059773829%


In [8]:
# Saving the model

# saving and loading a trained model
# from pathlib import Path

# # 1. create models directory
# MODEL_PATH=Path("models")
# MODEL_PATH.mkdir(parents=True,exist_ok=True)

# 2. create model save path
# MODEL_NAME="01_gesture_detection.pth"
# MODEL_SAVE_PATH=MODEL_PATH / MODEL_NAME

# # 3. save the model state dict

# print(f"Saving model to: {MODEL_SAVE_PATH}")
# torch.save(obj=model_1.state_dict(),f=MODEL_SAVE_PATH)

############################# YOU CAN START DIRECTLY FROM HERE BY LOADING THE MODEL.#################################

In [9]:


# -------------------------- NEXT TIME YOU CAN START FROM HERE BY LOADING THE MODEL DIRECTLY-----------------


# load a pytorch model

# create a new instace of linear regression model V2

# loaded_GestureModel_1=GestureModel(42,8)

# # load the saved model state_dict
# loaded_GestureModel_1.load_state_dict(torch.load(MODEL_SAVE_PATH))

# # put the loaded model to device
# loaded_GestureModel_1.to(device)

def detectGes(X, orgX, orgY):
  # print(X.shape)
  # processing data to make it like the data we train on
  # print(X)

  for id, cood in enumerate(X[0]):
    if (id % 2 == 0):
      # X[0][id]=torch.round((cood-orgX),decimals=3)
      X[0][id] = (cood-orgX)
    else:
      # X[0][id]=torch.round((cood-orgY),decimals=3)
      X[0][id] = (cood-orgY)

  # print(X)

  maxVal=abs(X[0][0])
  for val in X[0]:
    if(abs(val)>abs(maxVal)):
      maxVal=val
  
  # print(maxVal)
  
  # for id,cood in enumerate(X[0]):
  #   X[0][id]=torch.round((X[0][id]/abs(maxVal)),decimals=3)
  
  # X[0]=torch.round(X[0]/abs(maxVal),decimals=3)
  X[0]=X[0]/abs(maxVal)
  # print(maxVal)
  # print(X)

  return torch.softmax(model_1(X), dim=1).argmax(dim=1)


In [10]:

def preprocess(data):
    # first make all landmarks coordinate relative to zero
    # then normalize it to the maximum value
    # print(data)
    for i in range(len(data)):
        orgX = data[i][0]
        orgY = data[i][1]
        for id, cood in enumerate(data[i]):
            if id < 42:
                if (id % 2 == 0):
                    data[i][id] = round((cood-orgX), 3)
                else:
                    data[i][id] = round((cood-orgY), 3)
            else:
                break

        # print(data)
        maxVal = abs(data[i][0])
        # print(f"inital: {maxVal}")
        for idd, val in enumerate(data[i]):
            if idd < 42:
                if (abs(val) > abs(maxVal)):
                    maxVal = val
            else:
                break
        # print(f"final: {maxVal}")
        for id, cood in enumerate(data[i]):
            if id < 42:
                data[i][id] = round((data[i][id]/abs(maxVal)), 3)
    
    # print(data)


In [11]:
# testing detectGesture function

# ges=detectGes(X_test[0].unsqueeze(dim=0))

In [16]:
import cv2
import numpy as np
import mediapipe as mp
import time
from djitellopy import Tello
from threading import Thread, Lock, Event

left_right_vel=0
forward_backward_vel=0
up_down_vel=0
land=False
gesture_event=Event()


# drone=Tello()
# drone.connect()
# print(drone.get_battery())
# start streaming video from
# drone.streamon()

'''
In some cases, when working with tensors, the values of the tensor are not actually stored in memory, but are calculated on-the-fly as needed. 
When you assign a variable to an element of a tensor, you may be getting a reference to a calculation rather than a value stored in memory. 
As a result, changes to the tensor may affect the value of the variable even if the variable was not explicitly reassigned.

One way to avoid this behavior is to create a copy of the tensor using the clone() method before assigning it to a variable.
'''

lock=Lock()
# a thread-function to control the drone movement
def show_movement():

    count=1
    if(count==1):
        # drone.takeoff()
        time.sleep(1)
        # drone.move("up",20)

    while(True):
        # wait for the gesture update
        gesture_event.wait()  
        with lock:

            landT=land
            lrv=left_right_vel
            fbv=forward_backward_vel
            udv=up_down_vel

            if(landT==True and lrv==0 and fbv==0 and udv==0):
                # drone.land()
                time.sleep(1)
                break
            else:    
                # drone.send_rc_control(lrv,fbv,udv,0)
                pass

        gesture_event.clear()

# for mediapipe hands detection setup,mandatory
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1)
mpDraw = mp.solutions.drawing_utils

pTime = 0
cTime = 0

ges = -1

cap = cv2.VideoCapture(0)

# drone.takeoff()
# time.sleep(5)

# imgTemp=drone.get_frame_read()

# creating a thread to run the fucntion
show_movement_thread=Thread(target=show_movement,daemon=True)
show_movement_thread.start()

while True:

    success, img = cap.read()
    # img = imgTemp.frame
    img = cv2.flip(img, 1)
    # img = cv2.resize(img, (360, 240))
    # have to convert our image to rgb for detection
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)

    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            data = np.array([])
            for id, lm in enumerate(handLms.landmark):
                # print(id, lm)
                h, w, c = img.shape
                # data = np.append(data, round(int(lm.x*w), 3))
                # data = np.append(data, round(int(lm.y*h), 3))
                data = np.append(data, (lm.x*w))
                data = np.append(data, (lm.y*h))

            data1=data
            data1=data1.reshape(1,42)
            data = torch.from_numpy(data).type(torch.float).to(device)
            data=data.unsqueeze(dim=0)
            ges = detectGes(data,data[0][0].clone(),data[0][1].clone())
            # print("a\n")
            # preprocess(data1)
            # print(data,data1)
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
            
    with lock:
        if (ges == 0):
            cv2.putText(img, "FORWARD", (10, 170),
                    cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0), 3)
            forward_backward_vel=15
            up_down_vel=0
            left_right_vel=0
            gesture_event.set()

        elif (ges == 1):
            cv2.putText(img, "STOP", (10, 170),
                    cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 3)
            up_down_vel=0
            left_right_vel=0
            forward_backward_vel=0
            gesture_event.set()

        elif (ges == 2):
            cv2.putText(img, "UP", (10, 170),
                    cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0), 3)
            up_down_vel=-15
            left_right_vel=0
            forward_backward_vel=0
            gesture_event.set()

        elif (ges == 3):
            cv2.putText(img, "LAND", (10, 170),
                    cv2.FONT_HERSHEY_PLAIN, 3, (0, 0, 255), 3)
            up_down_vel=0
            left_right_vel=0
            forward_backward_vel=0
            land=True
            gesture_event.set()

        elif (ges == 4):
            cv2.putText(img, "DOWN", (10, 170),
                    cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0), 3)
            up_down_vel=15
            left_right_vel=0
            forward_backward_vel=0
            gesture_event.set()

        elif (ges == 5):
            cv2.putText(img, "BACK", (10, 170),
                    cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0), 3)
            forward_backward_vel=-15
            left_right_vel=0
            up_down_vel=0
            gesture_event.set()

        elif (ges == 6):
            cv2.putText(img, "LEFT", (10, 170),
                    cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0), 3)
            left_right_vel=-15
            forward_backward_vel=0
            up_down_vel=0
            gesture_event.set()

        elif (ges == 7):
            cv2.putText(img, "RIGHT", (10, 170),
                    cv2.FONT_HERSHEY_PLAIN, 3, (255,0,0), 3)
            left_right_vel=15
            forward_backward_vel=0
            up_down_vel=0
            gesture_event.set()
        else:
            cv2.putText(img," ",(10,170),cv2.FONT_HERSHEY_PLAIN,3,(0,255,0),3)

    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    # fps = 0
    cv2.putText(img, str(int(fps)), (10, 70),
                cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 0), 3)

    cv2.imshow('Image', img)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q') or land:
        cv2.destroyWindow('Image')
        with lock:
            left_right_vel=0
            forward_backward_vel=0
            up_down_vel=0
            land=True
            
        cap.release()
        # time.sleep(1)
        # drone.streamoff()
        break
    # time.sleep(5)
    


QObject::moveToThread: Current thread (0x86b52c0) is not the object's thread (0x9174ea0).
Cannot move to target thread (0x86b52c0)

QObject::moveToThread: Current thread (0x86b52c0) is not the object's thread (0x9174ea0).
Cannot move to target thread (0x86b52c0)

QObject::moveToThread: Current thread (0x86b52c0) is not the object's thread (0x9174ea0).
Cannot move to target thread (0x86b52c0)

QObject::moveToThread: Current thread (0x86b52c0) is not the object's thread (0x9174ea0).
Cannot move to target thread (0x86b52c0)

QObject::moveToThread: Current thread (0x86b52c0) is not the object's thread (0x9174ea0).
Cannot move to target thread (0x86b52c0)

QObject::moveToThread: Current thread (0x86b52c0) is not the object's thread (0x9174ea0).
Cannot move to target thread (0x86b52c0)

QObject::moveToThread: Current thread (0x86b52c0) is not the object's thread (0x9174ea0).
Cannot move to target thread (0x86b52c0)

QObject::moveToThread: Current thread (0x86b52c0) is not the object's thread